# KKT Callbacks with ConicIP

The interior point solver can sometimes be sped up dramatically by exploiting the simultaneous structure of Q, A and G. Here we will consider the simple problem:

minimize  ½xᵀQx - bᵀx,   s.t.  x ≥ 0.

The solver can be called with no special parameters,

In [ ]:
using ConicIP
using SparseArrays, LinearAlgebra

n = 1000

Q = sprandn(n, n, 0.1); Q = Q'*Q;
c = ones(n, 1);
A = sparse(1.0I, n, n);
b = zeros(n, 1);
𝐾 = [("R", n)];

@time conicIP(Q, c, A, b, 𝐾, verbose=true);

The speed of the solver is reasonable, as the default solver exploits the sparsity of the constraint matrix. We can do better, however.

___

## KKT Callbacks
To speed up the solver, we require a function `kktsolver` which returns a function `solve3x3gen` which, in turn returns a function solving the KKT system

    ┌            ┐ ┌   ┐   ┌   ┐
    │ Q  Gᵀ  Aᵀ │ │ a │   │ x │
    │ G         │ │ c │ = │ z │
    │ A    -FᵀF │ │ b │   │ y │
    └            ┘ └   ┘   └   ┘

Where F is a `Block` matrix with blocks corresponding to the cones specified in 𝐾. The block F[i] is a `Diagonal` for "R" cones, a `SymWoodbury` for "Q" cones, and a `VecCongurance` for "S" cones.

In this example, since we have no linear constraints, G is empty, and our KKT system is

    ┌            ┐ ┌   ┐   ┌   ┐
    │ Q      I   │ │ a │   │ x │
    │ I    -FᵀF  │ │ b │ = │ y │
    └            ┘ └   ┘   └   ┘

The system can be solved by pivoting on the second block:

    (Q + (FᵀF)⁻¹) a = x + (FᵀF)⁻¹ y,    b = (FᵀF)⁻¹ (a - y)

Because we only have polyhedral constraints, F⁻² is a diagonal matrix, thus the first equation is a diagonal perturbation to Q which can be solved via a Cholesky factorization.

In [ ]:
function kktsolver(Q, A, G, cone_dims)
    
    function solve3x3gen(F, F⁻¹)

      invFᵀF = inv(F'F)
      QpD⁻¹ = cholesky(Q + spdiagm(0 => (F[1].diag).^(-2)))

      function solve3x3(x, z, y)

        a = QpD⁻¹\(x + A'*(invFᵀF*y))
        b = invFᵀF*(y - A*a)
        c = zeros(0, 1)
        return(a, c, b)

      end

    end
    
end

@time sol = conicIP(Q, c, A, b, 𝐾, kktsolver=kktsolver; verbose=true);

This results in a significant improvement in speed and memory usage!

This pattern of pivoting on the third block happens often enough that we have encapsulated it in the convenience function `pivot`, which transforms a 2×2 solver of the system

    (Q + Aᵀ(FᵀF)⁻¹A) a = x,    G a = y

into a 3×3 solver. This is illustrated below

In [ ]:
function kktsolver2x2(Q, A, G, cone_dims)
    
  function solve3x3gen(F, F⁻¹)

    QpD⁻¹ = cholesky(Q + spdiagm(0 => (F[1].diag).^(-2)))
    return (y, x) -> (QpD⁻¹\y, zeros(0, 1))

  end
    
end

@time sol = conicIP(Q, c, A, b, 𝐾, kktsolver=pivot(kktsolver2x2); verbose=true);

And as a bonus we even get an extra boost in speed! 
___

In [ ]:
# Style cell removed - was using deprecated readall() API